In [2]:
import cv2
import face_recognition
import numpy as np
import os
import pickle
from datetime import datetime

# Initialize the webcam capture
cap = cv2.VideoCapture(0)

# Load the reference image (Elon Musk)
imgelon = face_recognition.load_image_file('elon.jpg')
imgelon_rgb = cv2.cvtColor(imgelon, cv2.COLOR_BGR2RGB)
face_locations = face_recognition.face_locations(imgelon_rgb)
train_elon_encodings = face_recognition.face_encodings(imgelon_rgb, face_locations)[0]

# Initialize a dictionary to store student names and their face encodings
student_encodings = {}

# Define the directory path where the student images are located
path = 'student_images'

# Load and encode student images
for filename in os.listdir(path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the full file path
        file_path = os.path.join(path, filename)

        # Load the image
        img = face_recognition.load_image_file(file_path)

        # Encode the face in the image
        face_locations = face_recognition.face_locations(img)
        face_encoding = face_recognition.face_encodings(img, face_locations)

        if len(face_encoding) > 0:
            # Use the filename (without the extension) as the student's name
            student_name = os.path.splitext(filename)[0]

            # Store the face encoding in the dictionary
            student_encodings[student_name] = face_encoding[0]

# Serialize the student encodings to a file using pickle
with open('student_encodings.pkl', 'wb') as output:
    pickle.dump(student_encodings, output)

# Function to mark attendance
def markAttendance(name):
    with open('Attendance.csv', 'a') as f:
        now = datetime.now()
        time = now.strftime('%H:%M:%S')
        date = now.strftime('%d-%B-%Y')
        f.write(f'{name}, {date}, {time}\n')

# Load pre-trained encodings and class names
with open('student_encodings.pkl', 'rb') as input:
    student_encodings = pickle.load(input)

while True:
    # Read a frame from the webcam
    success, img = cap.read()

    # Convert the frame to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces and encode them
    face_locations = face_recognition.face_locations(img_rgb)
    encoded_faces = face_recognition.face_encodings(img_rgb, face_locations)

    for encode_face, faceloc in zip(encoded_faces, face_locations):
        matches = face_recognition.compare_faces(list(student_encodings.values()), encode_face)
        faceDist = face_recognition.face_distance(list(student_encodings.values()), encode_face)
        matchIndex = np.argmin(faceDist)

        if matches[matchIndex]:
            name = list(student_encodings.keys())[matchIndex]
            y1, x2, y2, x1 = faceloc

            # Draw a rectangle around the recognized face
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            # Mark attendance
            markAttendance(name)

    # Display the webcam feed
    cv2.imshow('webcam', img)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
